In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from synthcity.utils.serialization import save_to_file, load_from_file

In [2]:
base_path = Path("/home/bcebere/Documents/datasets/ward")

train_static = pd.read_csv(base_path / "ward_static_train_data.csv.gz")
train_temporal = pd.read_csv(base_path / "ward_temporal_train_data_eav.csv.gz")

test_static = pd.read_csv(base_path / "ward_static_test_data.csv.gz")
test_temporal = pd.read_csv(base_path / "ward_temporal_test_data_eav.csv.gz")

train_static

,id,transfer,age,floor: MP2 MPU PR,floor: MP2 PERIOPERATIVE AREA,floor: RR 3F,floor: RR 5DR,floor: RR 5EMS,floor: RR 5EOB,floor: RR 5FDU IOF,...,race: Native Hawaiian or Other Pacific Islander,race: Other,race: Patient Refused,race: Unknown,race: White or Caucasian,cpt: Allogeneic stem cell transplantation,cpt: Autologous stem cell transplantation,cpt: Chemo,cpt: NA,icu_admission
0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2.0,1.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3.0,1.0,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,4.0,1.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,5.0,1.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4895,4896.0,1.0,47.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4896,4897.0,1.0,69.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4897,4898.0,1.0,88.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
4898,4899.0,0.0,43.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [3]:
train_temporal

,id,time,variable,value
0,1,0.0,DBP,107.0
1,1,0.0,SBP,174.0
2,1,0.0,Best Motor Response,5.0
3,1,0.0,Best Verbal Response,5.0
4,1,0.0,Eye Opening,3.0
...,...,...,...,...
9896383,4900,131.0,O2 Device: Simple mask,0.0
9896384,4900,131.0,O2 Device: Trach,0.0
9896385,4900,131.0,O2 Device: Trach Collar,0.0
9896386,4900,131.0,O2 Device: Transtracheal catheter,0.0


In [4]:
import numpy as np
import pandas as pd

def process_temporal(in_static_data, in_temporal_data):
    all_temporal_data = []
    horizons = []
    
    for uid in in_static_data["id"].unique():
        local_temporal_idx = in_temporal_data["id"] == uid
        local_temporal = in_temporal_data[local_temporal_idx]
        columns = sorted(local_temporal["variable"].unique())
        times = sorted(local_temporal["time"].unique())

        temporal_data = pd.DataFrame([], columns = columns)


        for horizon in times:
            local_temporal_horizon_idx = local_temporal["time"] == horizon
            local_temporal_horizon = local_temporal[local_temporal_horizon_idx]

            horizon_data = pd.DataFrame(-1 * np.ones((1, len(columns))), columns = columns)

            proc = local_temporal_horizon[["variable", "value"]]
            proc.index = local_temporal_horizon["variable"]
            proc = proc.drop(columns = ["variable"])
            proc = proc.T.reset_index(drop = True)
            print(len(proc.columns), len(columns))
            horizon_data[proc.columns] = proc

            temporal_data = pd.concat([temporal_data, horizon_data], ignore_index = True)
        temporal_data.index = times

        for col in columns:
            if col not in temporal_data:
                temporal_data[col] = -1
        horizons.append(temporal_data["time"])
        all_temporal_data.append(temporal_data[columns])
        
    assert len(all_temporal_data) == len(in_static_data)

    return all_temporal_data, horizons

proc_test_temporal, proc_test_horizons = process_temporal(test_static, test_temporal)
proc_train_temporal, proc_train_horizons = process_temporal(train_static, train_temporal)

38 38
112 38


ValueError: Columns must be same length as key